In [2]:
import numpy as np
import glob
import matplotlib.pyplot as plt
import cv2
import os
import torch
from src.utils.get_scene_bounds import get_scene_bounds

In [7]:
basepaths = [p for p in glob.glob('/home/rozenberszki/D_project/wsnsl/Datasets/Scannet/*') if not (p.endswith('.zip') or p.endswith('panoptic'))]
basepaths

['/home/rozenberszki/project/wsnsl/Datasets/Scannet/scene0693_00',
 '/home/rozenberszki/project/wsnsl/Datasets/Scannet/scene0427_00',
 '/home/rozenberszki/project/wsnsl/Datasets/Scannet/scene0221_01',
 '/home/rozenberszki/project/wsnsl/Datasets/Scannet/scene0616_00',
 '/home/rozenberszki/project/wsnsl/Datasets/Scannet/scene0645_02',
 '/home/rozenberszki/project/wsnsl/Datasets/Scannet/scene0144_01']

In [8]:
bound_file = '/home/rozenberszki/project/wsnsl/bounds.txt'
for basepath in basepaths:
    K = np.loadtxt(basepath + '/intrinsic/intrinsic_color.txt')
    depths_directory = basepath + '/depth/'
    depths = sorted(
        glob.glob(depths_directory + "*"),
        key=lambda x: int(x.split("/")[-1].split(".")[0]),
    )
    poses = np.loadtxt(basepath + '/traj_w_c.txt').reshape(-1, 4, 4)
    min, max = get_scene_bounds(depths, poses, K)
    with open(bound_file, 'a') as f:
        f.write(f"{basepath} \n {min} {max}\n{[[min[0], max[0]], [min[1], max[1]], [min[2], max[2]]]}\n\n")


0it [00:00, ?it/s]

866it [00:23, 36.36it/s]
659it [00:11, 56.76it/s]
780it [00:15, 49.88it/s]
758it [00:24, 30.33it/s]
726it [00:15, 47.24it/s]
678it [00:16, 40.56it/s]


In [12]:
import os
import shutil
basepaths = [p for p in glob.glob('/home/rozenberszki/project/wsnsl/Datasets/Scannet/*') if not (p.endswith('.zip') or p.endswith('panoptic') or p.endswith('scene0300_01'))]
# Specify the path to copy the content from
source_path = "/home/rozenberszki/project/wsnsl/configs/ScanNet/scene0300_01.yaml"

# Specify the path to create the YAML files
output_path = "/home/rozenberszki/project/wsnsl/configs/ScanNet/"

# Iterate over each path in basepaths
for path in basepaths:
    # Get the basename of the path
    basename = os.path.basename(path)
    
    # Create the output file path
    output_file = os.path.join(output_path, f"{basename}.yaml")
    
    # Copy the content from the source path to the output file
    shutil.copyfile(source_path, output_file)


In [27]:
''' H: 480 #480 # 720
  W: 640 #680 920
  fx: 476.333 #480
  fy: 476.333 #480
  cx: 318 # 318
  cy: 240 #  240'''
K = np.array([[476.333, 0.0, 318], [0.0, 476.333, 240], [0.0, 0.0, 1.0]])

In [29]:
for path in os.listdir('/home/rozenberszki/D_Project/wsnsl/Dataset/'):
    print(path)
    poses = np.loadtxt(os.path.join('/home/rozenberszki/D_Project/wsnsl/Dataset',path,'traj.txt')).reshape(-1, 4, 4)
    poses.shape
    directory_path = os.path.join('/home/rozenberszki/D_Project/wsnsl/Dataset',path,'color_path')

    # Retrieve and sort all PNG files numerically by the number in their filename
    depths = sorted(
        glob.glob(f'{directory_path}/*.png'), 
        key=lambda x: int(x.split('/')[-1].split('_')[1].split('.')[0])
    )

    # Display sorted file paths
    png_depth_scale = 1000
    depth_data = cv2.imread(depths[0], cv2.IMREAD_UNCHANGED)
    depth_data = depth_data.astype(np.float32) / png_depth_scale
    depth = np.array(depth_data)
    out = realWorldProject(np.array(np.where(depth > 0))[::-1], poses[0], K, torch.from_numpy(depth))
    temp= out[0:3, :]
    from tqdm import tqdm
    for i in tqdm(range(0,2000,10)):
        out = realWorldProject(np.array(np.where(depth > 0))[::-1], poses[i], K, torch.from_numpy(depth))
        temp = np.concatenate((temp, out[0:3, :]), axis=1)

    out_min = np.min(temp, axis=1)-1
    out_max = np.max(temp, axis=1)+1
    output=list(zip(out_min, out_max))
    asList=[]
    for i in output:
        asList.append(list(i))
    print(path, asList)

5fb5d2dbf2


100%|██████████| 200/200 [00:54<00:00,  3.67it/s]


5fb5d2dbf2 [[-3.022592696637439, 8.895336811511498], [-4.618500089949689, 6.148802828462057], [-7.540831599156962, 6.480087916397451]]
8b5caf3398


100%|██████████| 200/200 [00:42<00:00,  4.72it/s]


8b5caf3398 [[-2.919554124365529, 5.977171390725077], [-2.7563677767638715, 3.9751238634494404], [-6.215323185517098, 2.968945879673896]]
98b4ec142f


...

b20a261fdf [[-2.721766720031403, 12.863155382095279], [-4.187328205529724, 6.7582327172962], [-9.663427104574534, 6.541482654411162]]
56a0ec536c


100%|██████████| 200/200 [01:00<00:00,  3.30it/s]


56a0ec536c [[-4.070793785694184, 4.856251816133856], [-2.4926024940348896, 4.1097323130626515], [-5.196354947093756, 4.103471723382164]]
fe1733741f


100%|██████████| 200/200 [00:58<00:00,  3.41it/s]

fe1733741f [[-11.870805646465676, -0.08686367641626536], [-5.2383272854987215, 5.323754296533068], [-2.8142115403743437, 9.441466688237004]]


100%|██████████| 200/200 [00:32<00:00,  6.22it/s]


In [ ]:
'''import matplotlib.pyplot as plt

# Assuming 'out' is the variable containing the array
x = out[0]  # x-coordinates
y = out[1]  # y-coordinates
z = out[2]  # z-coordinates

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(x, y, z, s=2)

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

plt.show()
'''

In [ ]:
import matplotlib.pyplot as plt

[[-2.721766720031403, 12.863155382095279], [-4.187328205529724, 6.7582327172962], [-9.663427104574534, 6.541482654411162]]
